# Neural Networks Assignment: Norberto Rancharan


### For this assignment use data at: “https://www.kaggle.com/wendykan/lending-club-loan-data/download”


# Importing Necessary Libraries

In [1]:
# Import packages
import pandas as pd
import pickle
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import ensemble
from sklearn.externals import joblib

C:\Users\Norberto J Rancharan\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
pd.set_option('display.max_columns', 600)
pd.set_option('display.max_rows', 600)

# Pickle for Faster Reading In

In [3]:
model_data = pickle.load(open("model_data.p", "rb" ))

In [4]:
model_data

,loan_amount,funded_amount,investor_funds,installment,grade,emp_length,annual_income,loan_status,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,term_ 60 months,pymnt_plan_y,addr_state_AL,addr_state_AR,addr_state_AZ,addr_state_CA,addr_state_CO,addr_state_CT,addr_state_DC,addr_state_DE,addr_state_FL,addr_state_GA,addr_state_HI,addr_state_IA,addr_state_ID,addr_state_IL,addr_state_IN,addr_state_KS,addr_state_KY,addr_state_LA,addr_state_MA,addr_state_MD,addr_state_ME,addr_state_MI,addr_state_MN,addr_state_MO,addr_state_MS,addr_state_MT,addr_state_NC,addr_state_ND,addr_state_NE,addr_state_NH,addr_state_NJ,addr_state_NM,addr_state_NV,addr_state_NY,addr_state_OH,addr_state_OK,addr_state_OR,addr_state_PA,addr_state_RI,addr_state_SC,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,initial_list_status_w,application_type_Joint App,hardship_flag_Y,region_NorthEast,region_SouthEast,region_SouthWest,region_West,disbursement_method_DirectPay
0,2500,2500,2500.0,84.92,5,10.0,55000.0,0,18.24,0.0,1.0,NaN,45.0,9.0,1.0,4341,10.3,34.0,0.0,NaN,1,NaN,NaN,0.0,0.0,16901.0,2.0,2.0,1.0,2.0,2.0,12560.0,69.0,2.0,7.0,2137.0,28.0,42000.0,1.0,11.0,2.0,9.0,1878.0,34360.0,5.9,0.0,0.0,140.0,212.0,1.0,1.0,0.0,1.0,NaN,2.0,NaN,0.0,2.0,5.0,3.0,3.0,16.0,7.0,18.0,5.0,9.0,0.0,0.0,0.0,3.0,100.0,0.0,1.0,0.0,60124.0,16901.0,36500.0,18124.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
1,30000,30000,30000.0,777.23,4,10.0,90000.0,0,26.52,0.0,0.0,71.0,75.0,13.0,1.0,12315,24.2,44.0,0.0,NaN,1,NaN,NaN,0.0,1208.0,321915.0,4.0,4.0,2.0,3.0,3.0,87153.0,88.0,4.0,5.0,998.0,57.0,50800.0,2.0,15.0,2.0,10.0,24763.0,13761.0,8.3,0.0,0.0,163.0,378.0,4.0,3.0,3.0,4.0,NaN,4.0,NaN,0.0,2.0,4.0,4.0,9.0,27.0,8.0,14.0,4.0,13.0,0.0,0.0,0.0,6.0,95.0,0.0,1.0,0.0,372872.0,99468.0,15000.0,94072.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
2,5000,5000,5000.0,180.69,4,6.0,59280.0,0,10.51,0.0,0.0,NaN,NaN,8.0,0.0,4599,19.1,13.0,0.0,NaN,1,NaN,NaN,0.0,0.0,110299.0,0.0,1.0,0.0,2.0,14.0,7150.0,72.0,0.0,2.0,0.0,35.0,24100.0,1.0,5.0,0.0,4.0,18383.0,13800.0,0.0,0.0,0.0,87.0,92.0,15.0,14.0,2.0,77.0,NaN,14.0,NaN,0.0,0.0,3.0,3.0,3.0,4.0,6.0,7.0,3.0,8.0,0.0,0.0,0.0,0.0,100.0

In [5]:
model_data['Delinquent'] = model_data['loan_status']

In [6]:
model_data = model_data.drop(columns=['loan_status'],axis=1)


In [7]:
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline

# Original Data

X = model_data.loc[:, model_data.columns != 'Delinquent']
y = model_data.loc[:, model_data.columns == 'Delinquent']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                   test_size = 0.2,
                                                   shuffle = True,
                                                   random_state = 111)
print("Original data shapes: {}, {}".format(X_train.shape, X_test.shape))

Original data shapes: (1686024, 158), (421507, 158)


In [8]:
# Now we impute the missing values with SimpleImputer
# NN does not handle missing values well, therefore we must impute values where there are nulls

imputer = SimpleImputer(strategy='median')

# We fit the imputer to the train set
# The imputer wil learn the median of all variables
imputer.fit(X_train)

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='median', verbose=0)

In [9]:
# Now we impute the train and test set

X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

In [10]:
features = pd.read_csv('important_features.csv')
important_features = features['Features'].to_list()

In [11]:
important_features

['grade',
 'max_bal_bc',
 'acc_open_past_24mths',
 'hardship_last_payment_amount',
 'inq_last_6mths',
 'hardship_payoff_balance_amount',
 'open_il_24m',
 'dti',
 'loan_amount',
 'sec_app_inq_last_6mths',
 'orig_projected_additional_accrued_interest',
 'annual_income',
 'avg_cur_bal',
 'tot_hi_cred_lim',
 'term_ 60 months',
 'hardship_flag_Y',
 'bc_open_to_buy',
 'disbursement_method_DirectPay',
 'total_bc_limit',
 'emp_length',
 'investor_funds',
 'mths_since_recent_inq']

### Creating Copies of Test and Train of X 

In [12]:
model_data.loc[:, model_data.columns != 'Delinquent'].columns

Index(['loan_amount', 'funded_amount', 'investor_funds', 'installment',
       'grade', 'emp_length', 'annual_income', 'dti', 'delinq_2yrs',
       'inq_last_6mths',
       ...
       'addr_state_WV', 'addr_state_WY', 'initial_list_status_w',
       'application_type_Joint App', 'hardship_flag_Y', 'region_NorthEast',
       'region_SouthEast', 'region_SouthWest', 'region_West',
       'disbursement_method_DirectPay'],
      dtype='object', length=158)

In [13]:
X_test_copy = pd.DataFrame(X_test, columns=model_data.loc[:, model_data.columns != 'Delinquent'].columns).copy()
X_train_copy = pd.DataFrame(X_train,columns=model_data.loc[:, model_data.columns != 'Delinquent'].columns).copy()
y_test_copy = y_test.copy()
y_train_copy = y_train.copy()

### Using the Important Features Identified from Grid Search in the Ensemble project

In [14]:
X_trainNN = X_train_copy[important_features]
X_testNN = X_test_copy[important_features]

### Converting the Train and Test to Array to use in NN Model

In [15]:
X_testNN = X_testNN.values
X_trainNN = X_trainNN.values

In [16]:
y_trainNN = y_train_copy.values
y_testNN = y_test_copy.values

In [17]:
# Feature Scaling
min_max_sc = MinMaxScaler()
X_trainNN = min_max_sc.fit_transform(X_trainNN)
X_testNN = min_max_sc.transform(X_testNN)

In [18]:
X_testNN

array([[0.66666667, 0.00849433, 0.078125  , ..., 0.68421053, 0.125     ,
        0.2       ],
       [0.66666667, 0.00785599, 0.109375  , ..., 0.78947368, 0.2       ,
        0.        ],
       [0.5       , 0.00499307, 0.046875  , ..., 0.36842105, 0.15      ,
        0.48      ],
       ...,
       [0.66666667, 0.        , 0.0625    , ..., 0.26315789, 0.2       ,
        0.48      ],
       [0.16666667, 0.00617181, 0.21875   , ..., 0.47368421, 0.425     ,
        0.08      ],
       [0.83333333, 0.0060424 , 0.125     , ..., 0.68421053, 0.28      ,
        0.12      ]])

In [19]:
from tensorflow import keras
from tensorflow.python.keras import backend as k

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [21]:
# first step: create a Sequential object, as a sequence of layers. B/C NN is a sequence of layers.
classifier = Sequential()

# add the first hidden layer
classifier.add(Dense(units=5,kernel_initializer='glorot_uniform',
                    activation = 'relu'))

# add the second hidden layer
classifier.add(Dense(units=5,kernel_initializer='glorot_uniform',
                    activation = 'relu'))

# add the output layer
classifier.add(Dense(units=1,kernel_initializer='glorot_uniform',
                    activation = 'sigmoid'))

# compiling the NN
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

# train the model
classifier.fit(X_trainNN,y_trainNN,batch_size=10,epochs=20)

Train on 1686024 samples
Epoch 1/20
1686024/1686024 [==============================] - 196s 116us/sample - loss: 0.3555 - accuracy: 0.8698
Epoch 2/20
1686024/1686024 [==============================] - 181s 107us/sample - loss: 0.3541 - accuracy: 0.8700
Epoch 3/20
1686024/1686024 [==============================] - 177s 105us/sample - loss: 0.3537 - accuracy: 0.8701
Epoch 4/20
1686024/1686024 [==============================] - 182s 108us/sample - loss: 0.3533 - accuracy: 0.8700
Epoch 5/20
1686024/1686024 [==============================] - 184s 109us/sample - loss: 0.3527 - accuracy: 0.8700
Epoch 6/20
1686024/1686024 [==============================] - 182s 108us/sample - loss: 0.3525 - accuracy: 0.8701
Epoch 7/20
1686024/1686024 [==============================] - 190s 113us/sample - loss: 0.3523 - accuracy: 0.8701
Epoch 8/20
1686024/1686024 [==============================] - 193s 115us/sample - loss: 0.3522 - accuracy: 0.8701
Epoch 9/20
1686024/1686024 [==============================] - 1

In [22]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  115       
_________________________________________________________________
dense_1 (Dense)              multiple                  30        
_________________________________________________________________
dense_2 (Dense)              multiple                  6         
Total params: 151
Trainable params: 151
Non-trainable params: 0
_________________________________________________________________


In [27]:
# Predicting the Test set results
y_pred = classifier.predict(X_testNN)
y_pred

array([[0.16073012],
       [0.19235048],
       [0.16132253],
       ...,
       [0.16763553],
       [0.4547792 ],
       [0.12779778]], dtype=float32)

In [28]:
y_pred = (y_pred>0.5)

In [29]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_testNN, y_pred)
cm

array([[366543,     86],
       [ 54429,    449]], dtype=int64)

In [30]:
# k-fold clustering
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

Using TensorFlow backend.


In [31]:
def build_classifier():
    # first step: create a Sequential object, as a sequence of layers. B/C NN is a sequence of layers.
    classifier = Sequential()
    # add the first hidden layer
    classifier.add(Dense(units=5,kernel_initializer='glorot_uniform',
                    activation = 'relu'))
    # add the second hidden layer
    classifier.add(Dense(units=5,kernel_initializer='glorot_uniform',
                    activation = 'relu'))
    # add the output layer
    classifier.add(Dense(units=1,kernel_initializer='glorot_uniform',
                    activation = 'sigmoid'))
    # compiling the NN
    classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
    return classifier

In [32]:
classifier = KerasClassifier(build_fn=build_classifier, batch_size = 6, nb_epoch = 3)
accuracies = cross_val_score(estimator=best_classifier, X=X_trainNN, y = y_trainNN, cv=10, n_jobs=-1)

In [33]:
accuracies.mean()

0.8696222543716431

In [34]:
accuracies.std()

0.0007283766995042447

In [35]:
# Drop Out Regularization
# You can detect overfitting with the difference between test and train error or the high variance in cross-validation.

from tensorflow.keras.layers import Dropout

# first step: create a Sequential object, as a sequence of layers. B/C NN is a sequence of layers.
classifier = Sequential()

# add the first hidden layer
classifier.add(Dense(units=5,kernel_initializer='glorot_uniform',
                    activation = 'relu'))
classifier.add(Dropout(0.2))  #often start with 0.1, not solved go up

# add the second hidden layer
classifier.add(Dense(units=5,kernel_initializer='glorot_uniform',
                    activation = 'relu'))
classifier.add(Dropout(0.2))  #often start with 0.1, not solved go up

# add the output layer
classifier.add(Dense(units=1,kernel_initializer='glorot_uniform',
                    activation = 'sigmoid'))
# compiling the NN
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])

In [36]:
# fine tuning with Grid Search
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

def build_classifier(optimizer):
    # first step: create a Sequential object, as a sequence of layers. B/C NN is a sequence of layers.
    classifier = Sequential()
    # add the first hidden layer
    classifier.add(Dense(units=5,kernel_initializer='glorot_uniform',
                    activation = 'relu'))
    # add the second hidden layer
    classifier.add(Dense(units=5,kernel_initializer='glorot_uniform',
                    activation = 'relu'))
    # add the output layer
    classifier.add(Dense(units=1,kernel_initializer='glorot_uniform',
                    activation = 'sigmoid'))
    # compiling the NN
    classifier.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['acc'])
    return classifier

In [38]:
classifier_2 = KerasClassifier(build_fn=build_classifier)

# create a dictionary of hyper-parameters to optimize
parameters = {'batch_size':[10,25,32], 'nb_epoch':[1,2,5],'optimizer':['adam','rmsprop']}
grid_search = GridSearchCV(estimator = classifier_2, param_grid = parameters, scoring = 'accuracy', cv=10, n_jobs=-1)
grid_search = grid_search.fit(X_trainNN,y_trainNN)

best_parameters = grid_search.best_params_ 
best_accuracy = grid_search.best_score_

Train on 1686024 samples
1686024/1686024 [==============================] - 183s 108us/sample - loss: 0.3555 - acc: 0.8696


In [39]:
best_parameters

{'batch_size': 10, 'nb_epoch': 1, 'optimizer': 'adam'}

In [40]:
best_accuracy

0.8697462195081446

In [49]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
grid_predictions = grid_search.predict(X_testNN)

print('Confusion Matrix:')
print(confusion_matrix(y_testNN,grid_predictions))
print('\n')

print('Classification Report:')
print(classification_report(y_testNN,grid_predictions))
print('\n')

print('Accuracy: {}'.format(round(accuracy_score(y_testNN,grid_predictions),4)))

Confusion Matrix:
[[366568     61]
 [ 54626    252]]


Classification Report:
              precision    recall  f1-score   support

           0       0.87      1.00      0.93    366629
           1       0.81      0.00      0.01     54878

    accuracy                           0.87    421507
   macro avg       0.84      0.50      0.47    421507
weighted avg       0.86      0.87      0.81    421507



Accuracy: 0.8703


In [58]:
# first step: create a Sequential object, as a sequence of layers. B/C NN is a sequence of layers.
best_classifier = Sequential()

# add the first hidden layer
best_classifier.add(Dense(units=5,kernel_initializer='glorot_uniform',
                activation = 'relu'))

# add the second hidden layer
best_classifier.add(Dense(units=5,kernel_initializer='glorot_uniform',
                activation = 'relu'))

# add the output layer
best_classifier.add(Dense(units=1,kernel_initializer='glorot_uniform',
                activation = 'sigmoid'))

# compiling the NN
best_classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [59]:
# train the model on best params
best_classifier.fit(X_trainNN, y_trainNN, batch_size= 10, epochs= 1)

Train on 1686024 samples
1686024/1686024 [==============================] - 196s 116us/sample - loss: 0.3561 - accuracy: 0.8695


In [68]:
# Predicting the Test set results
y_pred = best_classifier.predict(X_testNN)
y_pred

array([[0.1129739 ],
       [0.15254194],
       [0.13239443],
       ...,
       [0.11847652],
       [0.46612746],
       [0.10612152]], dtype=float32)

In [69]:
best_classifier_predictions = pd.DataFrame(y_pred,columns=['Predictions'])

In [70]:
best_classifier_predictions

,Predictions
0,0.112974
1,0.152542
2,0.132394
3,0.101968
4,0.008065
...,...
421502,0.202482
421503,0.132105
421504,0.118477
421505,0.466127


## Save Model

In [71]:
best_classifier.save('LendinClub_NNclassifier_model.h5')

In [74]:
pd.DataFrame(best_classifier.predict_proba(X_testNN))

,0
0,0.112974
1,0.152542
2,0.132394
3,0.101968
4,0.008065
...,...
421502,0.202482
421503,0.132105
421504,0.118477
421505,0.466127


In [92]:
default_prob_NN = pd.DataFrame(index=y_test.index)
default_prob_NN['Actual'] = y_test['Delinquent'].to_list()
default_prob_NN['Prob_Default'] = best_classifier_predictions.iloc[:,0].to_list()

bins = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
labels = [0,10,20,30,40,50,60,70,80,90,100]
default_prob_NN['Buckets'] = pd.cut(default_prob_NN['Prob_Default'],bins= bins)
default_prob_NN.head(10)

,Actual,Prob_Default,Buckets
726380,0,0.112974,"(0.1, 0.2]"
1785001,0,0.152542,"(0.1, 0.2]"
364312,0,0.132394,"(0.1, 0.2]"
931731,0,0.101968,"(0.1, 0.2]"
344277,0,0.008065,"(0.0, 0.1]"
2201027,0,0.120401,"(0.1, 0.2]"
133993,0,0.071988,"(0.0, 0.1]"
26530,0,0.176436,"(0.1, 0.2]"
2101195,0,0.180724,"(0.1, 0.2]"
1689377,0,0.061733,"(0.0, 0.1]"


In [87]:
default_prob_NN['Buckets'].value_counts()

(0.0, 0.1]    213066
(0.1, 0.2]    148209
(0.2, 0.3]     42291
(0.3, 0.4]     13666
(0.4, 0.5]      3473
(0.5, 0.6]       549
(0.9, 1.0]       186
(0.6, 0.7]        58
(0.7, 0.8]         9
(0.8, 0.9]         0
Name: Buckets, dtype: int64

In [114]:
def_prob_NN = default_prob_NN.groupby('Buckets')['Actual'].apply(lambda x: x.value_counts()).unstack().fillna(0)
def_prob_NN.head()

,0,1
Buckets,,
"(0.0, 0.1]",199469.0,13597.0
"(0.1, 0.2]",125038.0,23171.0
"(0.2, 0.3]",31338.0,10953.0
"(0.3, 0.4]",8582.0,5084.0
"(0.4, 0.5]",1914.0,1559.0


In [115]:
round((def_prob_NN.iloc[:,1] / (def_prob_NN.iloc[:,0] + def_prob_NN.iloc[:,1]) * 100),3)

Buckets
(0.0, 0.1]      6.382
(0.1, 0.2]     15.634
(0.2, 0.3]     25.899
(0.3, 0.4]     37.202
(0.4, 0.5]     44.889
(0.5, 0.6]     52.459
(0.6, 0.7]     58.621
(0.7, 0.8]     66.667
(0.9, 1.0]    100.000
dtype: float64

In [116]:
def_prob_NN['Bad Rate %'] = round((def_prob_NN.iloc[:,1] / (def_prob_NN.iloc[:,0] + def_prob_NN.iloc[:,1]) * 100),3)
def_prob_NN.drop(columns=[0,1], inplace = True)
def_prob_NN.to_csv('def_prob_NN.csv')

,Bad Rate %
Buckets,
"(0.0, 0.1]",6.382
"(0.1, 0.2]",15.634
"(0.2, 0.3]",25.899
"(0.3, 0.4]",37.202
"(0.4, 0.5]",44.889
"(0.5, 0.6]",52.459
"(0.6, 0.7]",58.621
"(0.7, 0.8]",66.667
"(0.9, 1.0]",100.000


In [117]:
#def_prob_NN.to_csv('def_prob_NN.csv')